# Lattice-Based Tree Packing

Implement the egortrushin kernel approach:
- Start with 2 base trees in a specific configuration
- Translate them in x and y directions to create a grid pattern
- Use SA to optimize: base positions, base angles, translation vectors
- Validate with Shapely - NO overlaps allowed

In [ ]:
import numpy as np
import pandas as pd
import random
import math
import time
import json
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Libraries loaded")
print(f"Tree has {len(TX)} vertices")
print(f"Tree height: {TY.max() - TY.min():.2f}")
print(f"Tree width: {TX.max() - TX.min():.2f}")

In [ ]:
class ChristmasTree:
    """Represents a single Christmas tree with Shapely polygon."""
    def __init__(self, x=0.0, y=0.0, angle=0.0):
        self.x = float(x)
        self.y = float(y)
        self.angle = float(angle)
        self._update_polygon()
    
    def _update_polygon(self):
        angle_rad = np.radians(self.angle)
        cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
        rotated_x = TX * cos_a - TY * sin_a + self.x
        rotated_y = TX * sin_a + TY * cos_a + self.y
        self.polygon = Polygon(zip(rotated_x, rotated_y))
        self.vertices_x = rotated_x
        self.vertices_y = rotated_y
    
    def set_params(self, x, y, angle):
        self.x = float(x)
        self.y = float(y)
        self.angle = float(angle) % 360
        self._update_polygon()
    
    def clone(self):
        return ChristmasTree(self.x, self.y, self.angle)

def has_collision_shapely(trees):
    """Check for collisions using Shapely (RELIABLE)."""
    if len(trees) <= 1:
        return False
    for i in range(len(trees)):
        for j in range(i + 1, len(trees)):
            if trees[i].polygon.intersects(trees[j].polygon):
                if not trees[i].polygon.touches(trees[j].polygon):
                    intersection = trees[i].polygon.intersection(trees[j].polygon)
                    if intersection.area > 1e-10:
                        return True
    return False

def calculate_bounding_box(trees):
    all_x = np.concatenate([t.vertices_x for t in trees])
    all_y = np.concatenate([t.vertices_y for t in trees])
    return max(all_x.max() - all_x.min(), all_y.max() - all_y.min())

def calculate_score(trees):
    side = calculate_bounding_box(trees)
    return (side ** 2) / len(trees)

print("ChristmasTree class and helper functions defined")

In [ ]:
def generate_lattice_trees(x1, y1, a1, x2, y2, a2, dx_x, dx_y, dy_x, dy_y, nx, ny, n_target):
    """
    Generate trees using lattice/translation approach.
    
    (x1, y1, a1): position and angle of first base tree
    (x2, y2, a2): position and angle of second base tree
    (dx_x, dx_y): translation vector for x direction
    (dy_x, dy_y): translation vector for y direction
    (nx, ny): number of cells in x and y directions
    n_target: target number of trees
    
    Returns: list of ChristmasTree objects
    """
    trees = []
    
    for ix in range(nx):
        for iy in range(ny):
            # Translation offset for this cell
            offset_x = ix * dx_x + iy * dy_x
            offset_y = ix * dx_y + iy * dy_y
            
            # Tree 1 in this cell
            trees.append(ChristmasTree(x1 + offset_x, y1 + offset_y, a1))
            
            # Tree 2 in this cell
            trees.append(ChristmasTree(x2 + offset_x, y2 + offset_y, a2))
    
    return trees[:n_target]

print("Lattice generation function defined")

In [ ]:
class LatticeOptimizer:
    """
    Optimize lattice parameters using SA with Shapely validation.
    Parameters: x1, y1, a1, x2, y2, a2, dx_x, dx_y, dy_x, dy_y
    """
    def __init__(self, n_target, nx, ny, random_state=42):
        self.n_target = n_target
        self.nx = nx
        self.ny = ny
        random.seed(random_state)
        np.random.seed(random_state)
        
        # Initialize with a reasonable starting configuration
        # Two trees in interlocking pattern
        self.x1, self.y1, self.a1 = 0.0, 0.0, 0.0
        self.x2, self.y2, self.a2 = 0.4, 0.5, 180.0
        
        # Translation vectors - start with wide spacing
        self.dx_x, self.dx_y = 0.9, 0.0  # Horizontal translation
        self.dy_x, self.dy_y = 0.0, 0.7  # Vertical translation
    
    def generate_trees(self):
        return generate_lattice_trees(
            self.x1, self.y1, self.a1,
            self.x2, self.y2, self.a2,
            self.dx_x, self.dx_y, self.dy_x, self.dy_y,
            self.nx, self.ny, self.n_target
        )
    
    def get_params(self):
        return (self.x1, self.y1, self.a1, self.x2, self.y2, self.a2,
                self.dx_x, self.dx_y, self.dy_x, self.dy_y)
    
    def set_params(self, params):
        (self.x1, self.y1, self.a1, self.x2, self.y2, self.a2,
         self.dx_x, self.dx_y, self.dy_x, self.dy_y) = params
    
    def perturb(self, position_delta=0.02, angle_delta=3.0):
        old_params = self.get_params()
        
        choice = random.randint(0, 9)
        if choice == 0:
            self.x1 += random.uniform(-position_delta, position_delta)
        elif choice == 1:
            self.y1 += random.uniform(-position_delta, position_delta)
        elif choice == 2:
            self.a1 = (self.a1 + random.uniform(-angle_delta, angle_delta)) % 360
        elif choice == 3:
            self.x2 += random.uniform(-position_delta, position_delta)
        elif choice == 4:
            self.y2 += random.uniform(-position_delta, position_delta)
        elif choice == 5:
            self.a2 = (self.a2 + random.uniform(-angle_delta, angle_delta)) % 360
        elif choice == 6:
            self.dx_x += random.uniform(-position_delta, position_delta)
        elif choice == 7:
            self.dx_y += random.uniform(-position_delta, position_delta)
        elif choice == 8:
            self.dy_x += random.uniform(-position_delta, position_delta)
        elif choice == 9:
            self.dy_y += random.uniform(-position_delta, position_delta)
        
        return old_params
    
    def solve(self, nsteps=10000, Tmax=0.5, Tmin=0.0001, verbose=True):
        trees = self.generate_trees()
        
        if has_collision_shapely(trees):
            best_score = float('inf')
        else:
            best_score = calculate_score(trees)
        
        best_params = self.get_params()
        current_score = best_score
        
        T = Tmax
        cooling_rate = (Tmin / Tmax) ** (1.0 / nsteps)
        
        for step in range(nsteps):
            old_params = self.perturb()
            trees = self.generate_trees()
            
            if has_collision_shapely(trees):
                new_score = float('inf')
            else:
                new_score = calculate_score(trees)
            
            delta = new_score - current_score
            if delta < 0 or (new_score < float('inf') and random.random() < math.exp(-delta / T)):
                current_score = new_score
                if new_score < best_score:
                    best_score = new_score
                    best_params = self.get_params()
            else:
                self.set_params(old_params)
            
            T *= cooling_rate
            
            if verbose and step % 2000 == 0:
                print(f"Step {step}/{nsteps}, T={T:.6f}, Best={best_score:.6f}, Current={current_score:.6f}")
        
        self.set_params(best_params)
        return best_score, self.generate_trees()

print("LatticeOptimizer class defined")

In [ ]:
# Load baseline for comparison
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
baseline_df = pd.read_csv(baseline_path)

def get_baseline_score(n):
    prefix = f"{n:03d}_"
    trees_data = baseline_df[baseline_df['id'].str.startswith(prefix)]
    trees = []
    for _, row in trees_data.iterrows():
        x = float(str(row['x'])[1:])
        y = float(str(row['y'])[1:])
        deg = float(str(row['deg'])[1:])
        trees.append(ChristmasTree(x, y, deg))
    return calculate_score(trees), trees

# Test baseline scores for lattice N values
lattice_n_values = [72, 100, 110, 144, 156, 196, 200]
print("Baseline scores for lattice N values:")
for n in lattice_n_values:
    score, _ = get_baseline_score(n)
    print(f"N={n}: {score:.6f}")

In [ ]:
# Test lattice approach for N=72 (4x9x2 = 72)
print("\nTesting lattice approach for N=72...")

optimizer = LatticeOptimizer(n_target=72, nx=4, ny=9, random_state=42)

start_time = time.time()
lattice_score, lattice_trees = optimizer.solve(nsteps=15000, Tmax=0.3, Tmin=0.00001, verbose=True)
elapsed = time.time() - start_time

baseline_score_72, _ = get_baseline_score(72)
print(f"\nLattice score for N=72: {lattice_score:.6f}")
print(f"Baseline score for N=72: {baseline_score_72:.6f}")
print(f"Improvement: {baseline_score_72 - lattice_score:.6f}")
print(f"Time: {elapsed:.1f}s")

# Verify no overlaps
if has_collision_shapely(lattice_trees):
    print("WARNING: Lattice solution has overlaps!")
else:
    print("Lattice solution is VALID (no overlaps)")

In [ ]:
# Try multiple random seeds for N=72
print("\nTrying multiple seeds for N=72...")

best_score_72 = baseline_score_72
best_trees_72 = None

for seed in range(20):
    optimizer = LatticeOptimizer(n_target=72, nx=4, ny=9, random_state=seed)
    score, trees = optimizer.solve(nsteps=10000, Tmax=0.3, Tmin=0.00001, verbose=False)
    
    if score < best_score_72 and not has_collision_shapely(trees):
        best_score_72 = score
        best_trees_72 = trees
        print(f"Seed {seed}: NEW BEST {score:.6f}")

print(f"\nBest N=72 score: {best_score_72:.6f}")
print(f"Baseline N=72 score: {baseline_score_72:.6f}")
print(f"Improvement: {baseline_score_72 - best_score_72:.6f}")

In [ ]:
# Try all lattice N values
print("\nOptimizing all lattice N values...")

lattice_configs = {
    72: (4, 9),
    100: (5, 10),
    110: (5, 11),
    144: (6, 12),
    156: (6, 13),
    196: (7, 14),
    200: (7, 15),  # 210 trees, take first 200
}

results = {}

for n, (nx, ny) in lattice_configs.items():
    print(f"\n{'='*50}")
    print(f"Optimizing N={n} with grid {nx}x{ny}x2...")
    
    baseline_score, _ = get_baseline_score(n)
    print(f"Baseline score: {baseline_score:.6f}")
    
    best_score = baseline_score
    best_trees = None
    
    for seed in range(15):
        optimizer = LatticeOptimizer(n_target=n, nx=nx, ny=ny, random_state=seed)
        score, trees = optimizer.solve(nsteps=8000, Tmax=0.3, Tmin=0.00001, verbose=False)
        
        if score < best_score and not has_collision_shapely(trees):
            best_score = score
            best_trees = trees
            print(f"  Seed {seed}: NEW BEST {score:.6f}")
    
    improvement = baseline_score - best_score
    results[n] = {
        'baseline': baseline_score,
        'lattice': best_score,
        'improvement': improvement,
        'trees': best_trees
    }
    print(f"Final: {best_score:.6f}, Improvement: {improvement:.6f}")

In [ ]:
# Summary of lattice results
print("\n" + "="*60)
print("SUMMARY OF LATTICE APPROACH RESULTS")
print("="*60)

total_improvement = 0
for n in sorted(results.keys()):
    r = results[n]
    status = "✓ IMPROVED" if r['improvement'] > 0 else "✗ No improvement"
    print(f"N={n:3d}: Baseline={r['baseline']:.6f}, Lattice={r['lattice']:.6f}, Δ={r['improvement']:+.6f} {status}")
    if r['improvement'] > 0:
        total_improvement += r['improvement']

print(f"\nTotal improvement from lattice: {total_improvement:.6f}")

In [ ]:
# Create submission with improvements
print("\nCreating submission with improvements...")

# Start with baseline
submission_df = baseline_df.copy()

def format_val(v):
    return f"s{v}"

# Replace improved N values
for n, r in results.items():
    if r['improvement'] > 0 and r['trees'] is not None:
        prefix = f"{n:03d}_"
        # Remove old rows
        submission_df = submission_df[~submission_df['id'].str.startswith(prefix)]
        
        # Add new rows
        new_rows = []
        for i, tree in enumerate(r['trees']):
            new_rows.append({
                'id': f"{n:03d}_{i}",
                'x': format_val(tree.x),
                'y': format_val(tree.y),
                'deg': format_val(tree.angle)
            })
        submission_df = pd.concat([submission_df, pd.DataFrame(new_rows)], ignore_index=True)

# Sort
submission_df['sort_key'] = submission_df['id'].apply(lambda x: (int(x.split('_')[0]), int(x.split('_')[1])))
submission_df = submission_df.sort_values('sort_key').drop('sort_key', axis=1)

print(f"Submission has {len(submission_df)} rows")

In [ ]:
# Calculate total score
def calculate_total_score(df):
    total = 0
    for n in range(1, 201):
        prefix = f"{n:03d}_"
        trees_data = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in trees_data.iterrows():
            x = float(str(row['x'])[1:]) if str(row['x']).startswith('s') else float(row['x'])
            y = float(str(row['y'])[1:]) if str(row['y']).startswith('s') else float(row['y'])
            deg = float(str(row['deg'])[1:]) if str(row['deg']).startswith('s') else float(row['deg'])
            trees.append(ChristmasTree(x, y, deg))
        if len(trees) == n:
            total += calculate_score(trees)
    return total

new_total = calculate_total_score(submission_df)
baseline_total = calculate_total_score(baseline_df)

print(f"\nBaseline total score: {baseline_total:.6f}")
print(f"New total score: {new_total:.6f}")
print(f"Improvement: {baseline_total - new_total:.6f}")

In [ ]:
# Save submission
import os
os.makedirs('/home/submission', exist_ok=True)
submission_df.to_csv('/home/submission/submission.csv', index=False)
print("Saved submission to /home/submission/submission.csv")

# Save metrics
metrics = {
    'cv_score': new_total,
    'baseline_score': baseline_total,
    'improvement': baseline_total - new_total,
    'lattice_improvements': {str(n): r['improvement'] for n, r in results.items()}
}

with open('metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {new_total:.6f}")
print(f"Target: 68.919154")
print(f"Gap: {new_total - 68.919154:.6f}")